In [1]:
import pandas as pd
filepath = 'd:/大学/大三上/数据挖掘/作业/数据.xlsx'

In [2]:
data_combine = pd.read_excel(filepath)

In [3]:
data_combine_copy = data_combine.copy

In [4]:
import nltk
from nltk.corpus import stopwords

In [5]:
data_combine['word_split'] = data_combine.text.apply(lambda x:nltk.word_tokenize(str(x).lower()))

In [6]:
def dv_sa(x):
    a= ''
    for i in x:
        if i not in  stopwords.words('english'):
            i = '*' 
        a = a+' '+i
    return a

data_combine['dv_sa']=data_combine.word_split.apply(lambda x: dv_sa(x))

In [24]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [8]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1,5),analyzer='word',token_pattern='.*? .*?')
# 写一个正则匹配就行了

In [10]:
tfidf.fit(data_combine.dv_sa.values.tolist())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='.*? .*?', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
from sklearn.decomposition import TruncatedSVD

In [17]:
svd = TruncatedSVD(n_components=25,algorithm='arpack')

In [18]:
svd_matrix = svd.fit_transform(tfidf.transform(data_combine.dv_sa.values.tolist()))

In [19]:
print(svd_matrix)

[[ 0.41728845 -0.05932833 -0.00514919 ...,  0.01895476  0.06369608
  -0.03023372]
 [ 0.12905421 -0.03250405 -0.03129884 ..., -0.01946209  0.00700708
  -0.02436234]
 [ 0.39316539  0.01485718  0.02067984 ..., -0.00185167  0.02376045
  -0.02453574]
 ..., 
 [ 0.29190046  0.06774227 -0.04394066 ..., -0.01476449 -0.0065136
   0.03927061]
 [ 0.29888564 -0.06875855 -0.11935415 ..., -0.0308416   0.00662439
  -0.00910301]
 [ 0.35595373  0.02624089  0.00068273 ...,  0.02726556 -0.02396378
  -0.00951804]]


In [20]:
svd_feature = pd.DataFrame(svd_matrix)

In [21]:
svd_feature.columns = ['svd_word_1to5gram_'+str(i)for i in range(25)]

In [23]:
full_idf = tfidf.fit_transform(data_combine.dv_sa.values.tolist())
train_idf = tfidf.transform(train_data.dv_sa.values.tolist())
test_idf = tfidf.transform(test_data.dv_sa.values.tolist())

NameError: name 'train_data' is not defined

In [28]:
from sklearn.model_selection import KFold

In [44]:
svd_matrix_ = svd.fit_transform(tfidf_.transform(data_combine.dv_sa.values.tolist()))

In [25]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [26]:
full_idf = tfidf.fit_transform(data_combine.dv_sa.values.tolist())
train_idf = tfidf.transform(train_data.dv_sa.values.tolist())
test_idf = tfidf.transform(test_data.dv_sa.values.tolist())

In [29]:
import numpy as np
cv_score = []
pred_full_test = 0
pred_train = np.zeros([train_data.shape[0],3])
kf = KFold(n_splits=5,shuffle=True,random_state=2017)

In [30]:
from sklearn.naive_bayes import MultinomialNB
def run_by(train_x,train_y,test_x,test_y,test_x2):
    model = MultinomialNB()
    model.fit(train_x,train_y)
    pred_test_y = model.predict_proba(test_x)
    preb_test_y2 = model.predict_proba(test_x2)
    return pred_test_y,preb_test_y2,model

In [31]:
author = {'EAP':0,
         'HPL':1,
         'MWS':2}
train_y = train_data['author'].map(author)

In [32]:
from sklearn import metrics

In [34]:
for dev_index,val_index in kf.split(train_y):
    dev_x,val_x = train_idf[dev_index],train_idf[val_index]
    dev_y,val_y = train_y[dev_index],train_y[val_index]
    pred_val_y,pred_test_y,model = run_by(dev_x,dev_y,val_x,val_y,test_idf)
    pred_full_test+=pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_score.append(metrics.log_loss(val_y,pred_val_y))
print(cv_score)
pred_full_test/=5

[0.950374684619815, 0.93509830946931305, 0.93873820881282244, 0.95120680248155609, 0.95034412058227014]


In [134]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [35]:
train_data['eap_svd_word_dv_feature'] = pred_train[:,0]
train_data['hpl_svd_word_dv_feature'] = pred_train[:,1]
train_data['mws_svd_word_dv_feature'] = pred_train[:,2]
test_data['eap_svd_word_dv_feature'] = pred_full_test[:,0]
test_data['hpl_svd_word_dv_feature'] = pred_full_test[:,1]
test_data['mws_svd_word_dv_feature'] = pred_full_test[:,2]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [36]:
data_combine = pd.concat([train_data,test_data],axis=0)

In [39]:
for i in range(25):
    data_combine['svd_word_dv_1to5gram_'+str(i)] = svd_feature['svd_word_1to5gram_'+str(i)]

In [41]:
writer = pd.ExcelWriter('D:/大学/大三上/数据挖掘/作业/数据.xlsx')
data_combine.to_excel(writer,sheet_name='sheet1')
writer.save()